In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm

D:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = "D:/MLOntology/ontClassLabels_july2017.txt"
read_label(label_file)

In [3]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptPairList.append([splitted[1], splitted[2].replace("\r\n", ""), 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = "D:/workspace/MLDataProcessing/output/ontHierarchy_july2017.txt"
read_pair(pair_file)

first2pairs = conceptPairList[10:15]
print(first2pairs)
print(len(conceptPairList))

[['37225000', '52860004', 1], ['159386001', '159385002', 1], ['233836002', '233835003', 1], ['233836002', '304914007', 1], ['224923003', '224717003', 1]]
502206


In [4]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                conceptNotPairList.append([splitted[0], splitted[1].replace("\r\n", ""), 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = "D:/workspace/MLDataProcessing/output/taxNotPairs_july2017.txt"
read_not_pair(notPair_file)

# first2pairs = {k: conceptNotPairDict[k] for k in list(conceptNotPairDict)[10:15]}
first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

# In-place shuffle
random.shuffle(conceptNotPairList)

conceptNotPairList = conceptNotPairList[:len(conceptPairList)]

print(len(conceptNotPairList))

[['273187009', '272765000', 0], ['272877001', '272765000', 0], ['273216002', '272765000', 0], ['273125004', '272765000', 0], ['272973003', '272765000', 0]]
6166563
502206


In [5]:
path = "D:/MLOntology/model0"

model = gensim.models.Doc2Vec.load(path)

inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(model.docvecs.most_similar([inferred_vector], topn=10))

[('237267007', 0.5861426591873169),
 ('722912007', 0.5498865842819214),
 ('722913002', 0.5405725836753845),
 ('446466006', 0.5247948169708252),
 ('10759711000119103', 0.5159344673156738),
 ('267262008', 0.5109586715698242),
 ('10759661000119108', 0.5070662498474121),
 ('177130000', 0.5062016248703003),
 ('10759611000119105', 0.5009783506393433),
 ('12729009', 0.5008307695388794)]


In [ ]:
path = "D:/MLOntology/model1"

model = gensim.models.Doc2Vec.load(path)

inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(model.docvecs.most_similar([inferred_vector], topn=10))

In [6]:
feature_number = 1024

def readFromModel(id_pair_list, id_notPair_list, model):
    pair_list = id_pair_list + id_notPair_list
    random.shuffle(pair_list)
    list_ids = {}
    vector_list =[]
    label_list =[]
    for i, line in enumerate(pair_list):
        if line[0] in model.docvecs and line[1] in model.docvecs:
            a= model.docvecs[line[0]]
            b= model.docvecs[line[1]]
            c = np.array((a, b))
            list_ids[i] = (line[0], line[1])
    #         test_list.append(np.reshape(c, feature_number))
            vector_list.append(np.reshape(c, feature_number, order='F'))
            label_list.append(line[2])
    return list_ids, vector_list, label_list

list_ids, vector_list, label_list= readFromModel(conceptPairList, conceptNotPairList, model)

print(label_list[:20])

[1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(vector_list, label_list, test_size = 0.2)
print(X_train[:20])
print(X_train[:20])
print(y_train[:20])
print(y_test[:20])


[array([-0.0938481 , -0.26090142, -0.07325518, ...,  0.2134558 ,
       -0.08127657,  0.02696871], dtype=float32), array([-0.04491355, -0.05841781,  0.0730077 , ..., -0.18644407,
       -0.2568406 , -0.3720429 ], dtype=float32), array([-0.07458175, -0.19523714,  0.02509984, ..., -0.20835246,
       -0.05404728,  0.15774566], dtype=float32), array([ 0.03975468,  0.42562276, -0.08545467, ...,  0.39906406,
        0.45524302,  0.26211146], dtype=float32), array([-0.04149692, -0.10480236,  0.06591147, ...,  0.03666697,
        0.02502599,  0.03413479], dtype=float32), array([-0.03200416, -0.01477203, -0.01672264, ..., -0.0967933 ,
       -0.13597636, -0.12246318], dtype=float32), array([-0.10734122, -0.22792925, -0.14439787, ...,  0.08419887,
       -0.15326451, -0.21731113], dtype=float32), array([ 0.21762145,  0.1101729 , -0.17318687, ..., -0.18145218,
       -0.29609022,  0.02554329], dtype=float32), array([-0.10468335,  0.09955081,  0.08353478, ..., -0.128112  ,
       -0.18139517,  0.

In [10]:
#CNN
import numpy as np
import math
from math import sqrt

import tensorflow as tf


'''
In the data, there are 2 classes and every sample has 512 features
'''
# DATA_DIR = ''
CLASS_NUM = 2       #there are 2 classes
FEATURE_NUM = 1024   
TRAIN_ITER = 1500    #the number of iterations for training
display_step = 100        #how many iterations to display the results



train_feature = np.asarray(X_train)      #training features (list of list)
train_y = y_train        #training lables    (list)
test_feature = np.asarray(X_test)       #test features  (list of list)
test_y = y_test         #test labels    (list)


y_m = np.eye(2)[train_y]
test_y_m = np.eye(2)[test_y]

'''
y = wx+b        (vectors)
'''
#function to get variables 'w'
def weight_variable(shape, num):
    initial = tf.truncated_normal(shape, stddev=1/num)
    return tf.Variable(initial, name='weight')

#the bias 'b' in the equations
def bias_variable(shape, num):
    initial = tf.constant(0.0001, shape=shape)
    return tf.Variable(initial, name='bias')

#convolutional process
def conv1d(x, W):
    return tf.nn.conv1d(x, W, stride=1, padding='SAME')     #x: variable, w: weight, stride and padding (padding can be ignored currently) 

#pooling process
def max_pool_1x1(x, shape):
    x=tf.reshape(x,shape)       #it is transfered into four dimensions, but the other three are 1
    return tf.nn.max_pool(x, ksize=[1, 1, 4, 1],
                        strides=[1, 1, 2, 1], padding='SAME')

'''
The feature is 3 dimensional data.  [batch, length, channel] 
batch is usually ignored (for example there are 100 samples in a batch, so samples should not be modified mutually), length and channel are shown in the paper.
At first, the length is 512, and channel is 1.
Because our data are time series data, so length is enough, but for images, it may be [batch, length, width, channel]
'''
# the convolutional layer
def layer(features, f, input_n, channel, hidden_units, layer_index):
    """Construct a convolutional layer
    Args:
    features: Features placeholder, from the previous layer.
    f: the length
    input_n: Size of the features used in the convention.
    hidden_units: Size of the current hidden layer.
    layer_index: the index of layer
    Returns:
    hidden units: The unit output for the next layer.
    weights: the weights in the current hidden layer
    """
    # Hidden 1
    with tf.name_scope('hidden'+str(layer_index)) as scope:     # name scope may be ignored first
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, channel, hidden_units], math.sqrt(f))

        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(f))
    hidden = relu(conv1d(features, weights) + biases, 0.01)
    shape = [-1,1,f,hidden_units]
    h_pool1 = max_pool_1x1(hidden,shape)
    return h_pool1, weights

# fully connected layer, here the data are two dimension, [batch, length]
def densely_connect(features, input_n, hidden_units):
    """Construct a fully (densely) connected layer.
    Args:
    features: Features placeholder, from the previous layer.
    input_n: Size of units in the previous layer.
    hidden_units: Size of the current hidden layer.
    Returns:
    logits: The estimated output in last layer.
    weights: the weights in the hidden layer
    """
    with tf.name_scope('softmax_linear') as dense:
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, hidden_units], math.sqrt(input_n))
        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(input_n))
    logits = relu(tf.matmul(features, weights) + biases, 0.01)      # the matrix product operation
    return logits, weights

# dropout layer (it is not necessary)
# randomly set (1-keep_prob) percentage of units to be zero
def dropout(features, input_n, hidden_units, keep_prob):
    with tf.name_scope('dropout'):
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, hidden_units], math.sqrt(input_n))
        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(input_n))
    h_fc1_drop = tf.nn.dropout(features, keep_prob)
    drop_out = relu(tf.matmul(features, weights) + biases, 0.01)
    return drop_out

# calculate the loss in the neural network
def loss(logits, labels):
    """Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size, NUM_CLASSES].
    Returns:
    loss: Loss tensor of type float.
    """
    with tf.name_scope("Loss"):
        labels = tf.to_int64(labels)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=labels, logits=logits, name='xentropy')
    # tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
    return tf.reduce_mean(cross_entropy, name='xentropy_mean')

def next_batch(data, label, num):
    """Generate the next batch randomly
    Args:
    data: training data.
    label: training label.
    num: the size in a batch
    Returns:
    next batch's training features and labels.
    """
    index = np.arange(len(data))
    np.random.shuffle(index)
#     train_feature = data[np.array(index)[0:num]]
#     train_label = label[np.array(index)[0:num]]
#     return train_feature, train_label
    train_feature_batch = [data[b] for b in index[0:num]]
    train_feature_batch = np.asarray(train_feature_batch)
    train_label_batch = [label[b] for b in index[0:num]]
    train_label_batch = np.asarray(train_label_batch)
    return train_feature_batch, train_label_batch

def relu(x, alpha=0., max_value=None):
    '''ReLU.
    alpha: slope of negative section.
    '''
    negative_part = tf.nn.relu(-x)
    x = tf.nn.relu(x)
    if max_value is not None:
        x = tf.clip_by_value(x, tf.cast(0., dtype=tf.float32),
                             tf.cast(max_value, dtype=tf.float32))
    x -= tf.constant(alpha, dtype=tf.float32) * negative_part
    return x

#define a session to run the model
sess = tf.InteractiveSession()

#place holders for training features and label
#None means the value is variable
x = tf.placeholder(tf.float32, shape=[None, FEATURE_NUM])
y_ = tf.placeholder(tf.float32, shape=[None, CLASS_NUM])

# decide whether it is training or testing, it is not used in our model, but it may be used
is_training = tf.placeholder(tf.bool)

#from [-1, 512, 1] -> [-1, 256, 32] -> [-1, 128, 64] -> [-1, 64, 64] -> [-1, 32, 64] -> [-1, 16, 64] -> [-1, 8, 64] -> [-1, 200]

#6 hidden layers
x_1 = tf.reshape(x, [-1,FEATURE_NUM,1])
h_pool0, w0 = layer(x_1, FEATURE_NUM, 15, 1, 32, 0)
h_pool0 = tf.reshape(h_pool0, [-1,512,32])
h_pool1, w1 = layer(h_pool0, 512, 10, 32, 64, 1)
h_pool1 = tf.reshape(h_pool1, [-1,256,64])
h_pool2, w2 = layer(h_pool1, 256, 10, 64, 64, 2)
h_pool2 = tf.reshape(h_pool2, [-1,128,64])
h_pool3, w3 = layer(h_pool2, 128, 10, 64, 64, 3)
h_pool3 = tf.reshape(h_pool3, [-1,64,64])
h_pool4, w4 = layer(h_pool3, 64, 5, 64, 64, 4)
h_pool4 = tf.reshape(h_pool4, [-1,32,64])
h_pool5, w5 = layer(h_pool4, 32, 5, 64, 64, 5)
h_pool5 = tf.reshape(h_pool5, [-1,16,64])
h_pool6, w6 = layer(h_pool5, 16, 5, 64, 64, 6)
h_pool6 = tf.reshape(h_pool6, [-1,8,64])

#densely connected: 200 units
h_pool_flat = tf.reshape(h_pool6, [-1, 8*64])
h_dc, w_d = densely_connect(h_pool_flat, 8*64, 200)

#dropout
keep_prob = tf.placeholder(tf.float32)
y_conv=dropout(h_dc, (int)(h_dc.get_shape()[1]), CLASS_NUM, keep_prob)


beta = 0.001
cross_entropy = loss(y_conv, y_)
loss = cross_entropy +beta*(tf.nn.l2_loss(w0) + tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(w3)+tf.nn.l2_loss(w4)+tf.nn.l2_loss(w5)+tf.nn.l2_loss(w6)+tf.nn.l2_loss(w_d))  #L2 regularization
epsilon = 1e-5      # learning rate
train_step = tf.train.AdamOptimizer(epsilon).minimize(loss)     #optimization function, our goal is to minimize the loss

predict = tf.argmax(y_conv,1)   #the predicted class

# calculate the accuray, the corrected classified divided by the total size
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#saver to save the training check point
# variables can be restored in a new model by 'saver.restore(sess, save_path)'
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())  #initialize the variables


for i in range(1,TRAIN_ITER):       #training iterations
    d, l = next_batch(train_feature, y_m, 100)      # get 1000 samples in one batch
#     print("d size is %s, l size is %s "% (len(d), len(l)))
#     print("d size is %s, l size is %s "% (len(d[0]), len(l[0])))
    _, ls=sess.run([train_step,cross_entropy], feed_dict={x: d, y_: l, keep_prob: 1, is_training:True})     #run the train step (optimization function), the second one is just to show the loss in this iteration.   THE FEED dictionary is to feed the place holders which are needed in the optimization function.
    
    if i%display_step==0:
        print(_, i)
        acc = sess.run([accuracy], feed_dict={x: d, y_: l, keep_prob: 1, is_training:False})
        print("Train Loss:", ls, "Acc:", acc)

# sess.run  or tensor.eval are two ways
# get the accuracy in the testing data
# need to cut down the size of testing data, mihgt be in batches

print(accuracy.eval(session=sess, feed_dict={x:test_feature, y_:test_y_m, keep_prob: 1, is_training:False}))


# save the model results
save_path = saver.save(sess, "./model-noleaky.ckpt")
print("Model saved in file: %s" % save_path)

print("done")


None 10
Train Loss: 0.70140177 Acc: [0.45]
None 20
Train Loss: 0.68430257 Acc: [0.56]
None 30
Train Loss: 0.69042 Acc: [0.49]
None 40
Train Loss: 0.6822358 Acc: [0.63]
None 50
Train Loss: 0.6847766 Acc: [0.51]
None 60
Train Loss: 0.6735382 Acc: [0.58]
None 70
Train Loss: 0.6727262 Acc: [0.51]
None 80
Train Loss: 0.66415274 Acc: [0.54]
None 90
Train Loss: 0.6712666 Acc: [0.57]


ResourceExhaustedError: OOM when allocating tensor with shape[200877,1,1024,32]
	 [[Node: conv1d_7/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/cpu:0"](conv1d_7/ExpandDims, conv1d_7/ExpandDims_1)]]

Caused by op 'conv1d_7/Conv2D', defined at:
  File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Anaconda\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File

In [ ]:

y_pred = sess.run(predict, feed_dict={x:test_feature, keep_prob:1, is_training:False})
print(y_pred[:20])
print(test_y[:20])


err_ids=np.flatnonzero(y_pred != test_y)

print(err_ids)
print(err_ids.size)
for err_id in err_ids:
    print("index %d predicted label %s, but true label is %s" % (err_id, y_pred[err_id], test_y[err_id]))
    idpair = test_list_ids[err_id] 
    concept1 = conceptLabelDict[idpair[0]]
    concept2 = conceptLabelDict[idpair[1]]
    print("%s Concept Pairs: (%s --- %s)" % (idpair, concept1, concept2 ))


In [ ]:
result = y_pred
test_label_list = test_y

from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
print(accuracy_score(result, test_label_list))
print(average_precision_score(result, test_label_list))

print(f1_score(result, test_label_list, average='macro') ) 

print(f1_score(result, test_label_list, average='micro')  )

print(f1_score(result, test_label_list, average='weighted') )

print(f1_score(result, test_label_list, average=None))

print(precision_score(result, test_label_list, average=None))
print(recall_score(result, test_label_list, average=None))

print(roc_auc_score(result, test_label_list, average=None))